# What is a Context Manager?
A context manager is a Python object that sets up a resource before a block of code runs, and cleans it up automatically when the block finishes.

👉 The most common way to use them is with the with statement.

## 1. The `with` Statement (Built-in File Context Manager)

When working with files, `with` ensures the file is automatically closed after the block.


In [1]:
with open("example.txt", "w") as f:
    f.write("Hello, Prasanna!")
# File is automatically closed here

✅ No need to call `f.close()`.

👉 What happens here:
- open("example.txt", "w") → file is opened in write mode.
- `f` is the file object.
- Inside the with block, you can use the file (f.write(...)).
- Once the block ends → Python automatically calls f.close(), even if an error occurs.

✅ This is the simplest context manager, built into Python’s file handling.

✅ Even if an error occurs inside the block, Python closes the file safely.

## 2. Class-Based Context Manager (`__enter__` / `__exit__`)

We can build our own context manager for file handling.

In [2]:
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        print("Opening file...")
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        print("Closing file...")
        if self.file:
            self.file.close()

# Usage
with FileManager("example2.txt", "w") as f:
    f.write("Managed by class-based context manager")


Opening file...
Closing file...


👉 What happens here:
- When with `FileManager(...)` runs → Python calls `__enter__`.
- `__enter__` opens the file and returns it.
- Inside the block → you write to the file.
- When the block ends → Python automatically calls `__exit__`.
- `__exit__` ensures the file is closed.

✅ This is exactly what happens inside Python’s built-in file handling.

## 3. Function-Based Context Manager (@contextmanager)
- Python provides a shortcut using `contextlib.contextmanager`.
- Here, we use a generator with `yield`.

In [3]:
from contextlib import contextmanager

@contextmanager
def file_manager(filename, mode):
    print("Opening file...")
    f = open(filename, mode)
    try:
        yield f   # return control to the with-block
    finally:
        print("Closing file...")
        f.close()

with file_manager("example3.txt", "w") as f:
    f.write("Managed by function-based context manager")


Opening file...
Closing file...


👉 What happens here:
- Code before `yield` → setup (open file).
- The value from `yield` → given to the with block (f).
- Code after `yield` → cleanup (close file).
- `finally` ensures cleanup happens even if an error occurs.

✅ This approach is shorter and more Pythonic than writing a class.

## 4. Managing Multiple Files at Once

In [5]:
with open("example3.txt", "r") as f1, open("output.txt", "w") as f2:
    data = f1.read()
    f2.write(data)


👉 What’s happening:
- Two files are opened at the same time (f1, f2).
- The with statement ensures that both are closed automatically at the end of the block.
- Even if one file raises an exception, Python still makes sure both are cleaned up.

✅ Cleaner than writing nested with blocks.

## 5. Suppressing Exceptions (contextlib.suppress)

In [6]:
from contextlib import suppress

with suppress(FileNotFoundError):
    with open("non_existent.txt") as f:
        print(f.read())   # no error, just ignored


👉 What’s happening:
- Normally, opening a missing file → raises FileNotFoundError.
- With suppress(FileNotFoundError) → Python ignores this specific exception.
- The code continues without crashing.

✅ Use when certain errors are “ok” and you don’t want extra try/except noise.
(Example: deleting a temp file that may or may not exist.)

## 6. ExitStack → Managing Many Resources Dynamically

In [7]:
from contextlib import ExitStack

files = ["a.txt", "b.txt", "c.txt"]

with ExitStack() as stack:
    handles = [stack.enter_context(open(fname, "w")) for fname in files]
    for f in handles:
        f.write("Hello, Prasanna!\n")


👉 What’s happening:
- Suppose you have a list of files and you want to open all of them.
- You don’t know the number in advance (it could be 2 or 200).
- ExitStack dynamically manages them:
- `stack.enter_context(...)` opens each file.
- When the block ends → ExitStack closes all files automatically.

✅ Perfect when working with variable resources (files, sockets, DBs, etc.).

## 7. Async Context Managers (async with)

In [9]:
import asyncio
from contextlib import asynccontextmanager

@asynccontextmanager
async def my_async_file():
    print("Opening async resource...")
    yield "Async Resource"
    print("Closing async resource...")

async def main():
    async with my_async_file() as f:
        print(f)

#asyncio.run(main())


# In a Jupyter cell, do this:
await main()




Opening async resource...
Async Resource
Closing async resource...


👉 What’s happening:
- `asynccontextmanager` works like `@contextmanager`, but for `async`/`await` code.
- `async` `with` ensures that setup/cleanup also work in async functions.
- Very useful in FastAPI, aiohttp, and async DB drivers.

✅ Example: opening a database connection in an async web server.

### 🎯 Final Summary: Context Managers in Python

---

#### 1. The `with` Statement (Built-in)
- Simplest form of context manager, often used with files.
- Ensures resources (like files) are **automatically cleaned up** after use.
- Equivalent to writing `try/finally` blocks manually, but shorter and safer.

---

#### 2. Class-Based Context Managers
- Define `__enter__()` for setup (resource allocation).
- Define `__exit__()` for cleanup (resource release).
- Gives full control over resource management, including handling exceptions.

---

#### 3. Function-Based Context Managers (`@contextmanager`)
- Use `contextlib.contextmanager` and `yield` for a simpler syntax.
- Code before `yield` → setup.
- Code after `yield` → cleanup.
- More concise than writing classes, especially for simple managers.

---

#### 4. Why Context Managers?
- They eliminate the need for **manual cleanup** (like `close()`).
- Guarantee cleanup happens even if an **exception occurs**.
- Make code more **readable, concise, and reliable**.

---

#### 5. Multiple Context Managers
- Allows opening/using **multiple resources** in a single `with` statement.
- Ensures **all resources** are closed properly when the block exits.
- Cleaner than writing nested `with` statements.

---

#### 6. Suppressing Exceptions (`suppress`)
- Lets you **ignore specific exceptions** automatically.
- Useful when certain errors are expected (e.g., file may not exist).
- Prevents unnecessary `try/except` clutter in code.

---

#### 7. ExitStack
- Advanced tool for managing a **dynamic or unknown number of resources**.
- You can add resources as needed (`enter_context`).
- Automatically closes all resources at the end of the block.

---

#### 8. Async Context Managers (`async with`)
- Used in asynchronous programming with `async/await`.
- Ensures async resources (like DB connections, network sockets) are setup/cleaned properly.
- Essential in frameworks like **FastAPI, aiohttp, async DB drivers**.

---

#### ✅ Key Takeaway
Context managers = **automatic setup + cleanup**.  
They:
- Simplify resource handling,
- Prevent memory/resource leaks,
- Work in both synchronous and asynchronous worlds,
- Scale from simple file handling → to advanced use cases (DBs, threads, APIs).

Context managers are everywhere in Python — mastering them makes your code **cleaner, safer, and more professional**.
